In [1]:
import requests
import csv
import logging

logging.basicConfig(level=logging.INFO)

def fetch_data(url):
    """
    Fetch JSON data from the given URL.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Failed to fetch data from {url}: {e}")
        return None

def get_country_name(country_code, countries_data):
    """
    Get the country name from the country code.
    """
    for country in countries_data.get('countries', []):
        if country['code'] == country_code:
            return country['name']
    return None

def process_laureates_data(laureates_data, countries_data):
    """
    Process laureates data and extract required information.
    """
    processed_data = []
    for laureate in laureates_data['laureates']:
        id = laureate.get('id', '')
        name = laureate.get('firstname', '') + ' ' + laureate.get('surname', '') if laureate.get('surname') else laureate.get('firstname', '')
        dob = laureate.get('born', {}).get('date', '')
        unique_prize_years = ';'.join(set(prize['year'] for prize in laureate.get('prizes', [])))
        unique_prize_categories = ';'.join(set(prize['category'] for prize in laureate.get('prizes', [])))
        gender = laureate.get('gender', '')
        country_code = laureate.get('bornCountryCode', '')
        country_name = get_country_name(country_code, countries_data)
        
        processed_data.append({
            'id': id,
            'name': name,
            'dob': dob,
            'unique_prize_years': unique_prize_years,
            'unique_prize_categories': unique_prize_categories,
            'gender': gender,
            'bornCountry': country_name
        })
    return processed_data

def write_to_csv(data, filename):
    """
    Write data to CSV file.
    """
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['id', 'name', 'dob', 'unique_prize_years', 'unique_prize_categories', 'gender', 'bornCountry']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for row in data:
                writer.writerow(row)
        logging.info(f"Data successfully written to {filename}")
    except IOError as e:
        logging.error(f"Failed to write to CSV file: {e}")

def main():
    laureates_url = 'https://raw.githubusercontent.com/ashwath92/nobel_json/main/laureates.json'
    countries_url = 'https://raw.githubusercontent.com/ashwath92/nobel_json/main/countries.json'

    laureates_data = fetch_data(laureates_url)
    countries_data = fetch_data(countries_url)

    if laureates_data and countries_data:
        processed_data = process_laureates_data(laureates_data, countries_data)
        write_to_csv(processed_data, 'nobel_laureates.csv')
    else:
        logging.error("Failed to fetch required data. Exiting...")

if __name__ == "__main__":
    main()